In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
mc_data = pd.read_json('DataSets/SciFi_200000_Clean_TrackStudy.json')
real_data = pd.read_json('DataSets/SciFi_200000_AllHits.json')

In [ ]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1
# Data columns
i = 0
for val in real_data.columns:
    print(i," ",val)
    i = i + 1

In [ ]:
mc_hits = mc_data.values
real_hits = real_data.values

In [ ]:
# Create X and Y matrices for the future model
length = mc_hits.shape[0]

X = np.zeros((length, 1))
Y = np.zeros((length, 11))

i = 0
empty_particles = []
for i in range(0,length):
    
    if ((len(mc_hits[i][5]) > 1)):
        # Get the first element of the hit, in the first layer
        X[i] = mc_hits[i][5][0]
        
        # Get only the first element if there are several repeated with small differences
        j = 0
        extra_hits = mc_hits[i][5]
        try:
            while (np.abs(extra_hits[j] - extra_hits[j + 1]) < 5):
                j += 1
            j += 1
        except IndexError as e:
            X[i] = np.zeros(1)
            continue
        
        # Get all hits from the given particle which are not located in the first layer
        extra_hits = mc_hits[i][5][j:]
        actual_pos = 0
        j = 0
        try:
            while (j < len(extra_hits)):
                Y[i][actual_pos] = extra_hits[j]
                # Same as above, get only the first element in case there are several hits with similar X position
                while (j + 1 < len(extra_hits) and np.abs(extra_hits[j] - extra_hits[j + 1]) < 3):
                    j += 1
                actual_pos += 1
                j += 1
        except IndexError:
            X[i] = np.zeros(1)
            Y[i] = np.zeros(11)
            
# Removing empty values, as some hits in the mc_hits dataset do not contain readings (PrHit...)
Y = Y[Y[:,0] != 0]
X = X[X[:,0] != 0]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import metrics

model = Sequential()
model.add(Dense(12, input_dim=1, kernel_initializer='normal', activation='linear'))
model.add(Dense(8, activation='relu'))
model.add(Dense(11, activation='linear'))
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(X, Y, epochs=150, batch_size=50,  verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
def showPredictedTrack(X, Y):
    fig = plt.figure(figsize=(19, 8))
    ax = fig.add_subplot(111)
    ax.plot(X, Y)       
    ax.grid()
    plt.show()